## Testing CNN implentation with Keras

In [ ]:
# Pathing Libraries
from pathlib import Path
import os
import os.path as op
import glob

# Image Libraries
import nibabel as nib
import numpy as np
import imageio as iio
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

from tensorflow import keras


In [ ]:
## Pathing
path = '../'

images_path = Path(path, 'data', 'images')
masks_path = Path(path, 'data', 'masks')
segs_path = Path(path, 'data', 'segs_refs')
metadata_path = Path(path, 'data', 'metadata')
